In [0]:
import os
import math
import numpy as np
import torch
from torch.utils.data import Dataset
from torch.utils.data import random_split
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import transforms
from PIL import Image
import zipfile
import urllib.request
import os.path
from IPython.display import display

In [2]:
#4/rQHis9i6K34XFynk3YJcBeGtsficMpxWds--DqmDSd8pK-6ToIx8mIk
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
root_path = 'gdrive/My Drive/CheXnet/'  #change dir to your project folder

In [4]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"ofirit","key":"6f3bd89bc0908388f7ec8b5bc18763b8"}'}

In [5]:
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json  # set permission

kaggle.json


In [6]:
!kaggle datasets download nih-chest-xrays/sample -p /content/CheXnet

100% 4.19G/4.20G [01:33<00:00, 40.9MB/s]
100% 4.20G/4.20G [01:33<00:00, 47.9MB/s]


In [7]:
with zipfile.ZipFile('/content/CheXnet/sample.zip', 'r') as z:
    for f in z.filelist:
        print(f.filename)
        break

sample/images/00000013_005.png


In [0]:
class DataPreprocessing(Dataset):

    classEncoding = {
        'Atelectasis': torch.FloatTensor([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
        'Consolidation': torch.FloatTensor([0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
        'Infiltration': torch.FloatTensor([0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
        'Pneumothorax': torch.FloatTensor([0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
        'Edema': torch.FloatTensor([0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
        'Emphysema': torch.FloatTensor([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
        'Fibrosis': torch.FloatTensor([0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]),
        'Effusion': torch.FloatTensor([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]),
        'Pneumonia': torch.FloatTensor([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]),
        'Pleural_Thickening': torch.FloatTensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]),
        'Cardiomegaly': torch.FloatTensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]),
        'Nodule': torch.FloatTensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]),
        'Hernia': torch.FloatTensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]),
        'Mass': torch.FloatTensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]),
        'No Finding': torch.FloatTensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1])
    }
    
    def __init__(self,classEncoding=classEncoding):
        self.image_names = []
        self.labels = []
        with zipfile.ZipFile('/content/CheXnet/sample.zip', 'r') as z:
          f = z.open('sample_labels.csv','r')
          title = True
          for line in f:
            if (title):
              title = False
              continue
            items = str(line).split(",")
            image_name = items[0][2:]
            image_name = os.path.join("sample/images/", image_name)
            self.image_names.append(image_name)

            label = items[1]  # list of diseases
            diseases_list = label.split("|")
            labelTensor = torch.FloatTensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
            for disease in diseases_list:
              labelTensor = labelTensor.add(classEncoding[disease])
            self.labels.append(labelTensor)
          

            
    def __getitem__(self, index):
      image_path = self.image_names[index]
      with zipfile.ZipFile('/content/CheXnet/sample.zip', 'r') as z:
        image = Image.open(z.open(image_path,'r')).convert('RGB')
        normalize = transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        preprocess = transforms.Compose([
            transforms.Resize(256),
            transforms.TenCrop(224),
            transforms.Lambda
            (lambda crops: torch.stack([transforms.ToTensor()(crop) for crop in crops])),
            transforms.Lambda
            (lambda crops: torch.stack([normalize(crop) for crop in crops]))
        ])
        image = preprocess(image)
      return image, self.labels[index]
      

    def __len__(self):
        return len(self.image_names)



In [0]:
def compute_score_with_logits(logits, labels):
  logits = torch.max(logits, 1)[1].data # argmax
  one_hots = torch.zeros(*labels.size()).cuda()
  one_hots.scatter_(1, logits.view(-1, 1), 1)
  scores = (one_hots * labels)

  return scores
  
def tile(a, dim, n_tile):
  init_dim = a.size(dim)
  repeat_idx = [1] * a.dim()
  repeat_idx[dim] = n_tile
  a = a.repeat(*(repeat_idx))
  order_index = torch.LongTensor(np.concatenate([init_dim * np.arange(n_tile) + i for i in range(init_dim)]))
  return torch.index_select(a, dim, order_index)

def get_label_for_image(model, image_path):
  classes = ['Atelectasis', 'Consolidation', 'Infiltration', 'Pneumothorax', 'Edema', 'Emphysema', 'Fibrosis',
      'Effusion', 'Pneumonia', 'Pleural_Thickening', 'Cardiomegaly', 'Nodule', 'Hernia', 'Mass', 'No Finding']
  input_image_grey = Image.open(image_path)
  input_image = input_image_grey.convert('RGB')
  preprocess = transforms.Compose([transforms.Resize(256),
                                   transforms.CenterCrop(224),
                                   transforms.ToTensor(),
                                   transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
                                   ])
  input_tensor = preprocess(input_image)
  input_batch = input_tensor.unsqueeze(0)
  output = model(input_batch)
  index_tensor = torch.argmax(output)
  index = index_tensor.item()
  return classes[index]

In [0]:
class DenseNet121(nn.Module):
    def __init__(self):
        super(DenseNet121, self).__init__()
        self.model = torchvision.models.densenet121(pretrained=True)
        self.model.classifier = nn.Sequential(
            nn.Linear(1024, 15),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.model(x)
        return x

Train with DesNet

In [12]:
data = DataPreprocessing()
train_set, test_set = random_split(data, [math.ceil(len(data) * 0.8), math.floor(len(data) * 0.2)])

trainloader = torch.utils.data.DataLoader(train_set, batch_size=5, shuffle=True, num_workers=5)
testloader = torch.utils.data.DataLoader(test_set, batch_size=4, shuffle=False)

classes = ['Atelectasis', 'Consolidation', 'Infiltration', 'Pneumothorax', 'Edema', 'Emphysema', 'Fibrosis',
      'Effusion', 'Pneumonia', 'Pleural_Thickening', 'Cardiomegaly', 'Nodule', 'Hernia', 'Mass', 'No Finding']

model = DenseNet121().cuda()
model = nn.DataParallel(model).cuda()
# print(model)
# train = Train(train_set, model.cuda())

Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.cache/torch/checkpoints/densenet121-a639ec97.pth
100%|██████████| 32342954/32342954 [00:00<00:00, 97457443.85it/s]


In [13]:
%%time

criterion = nn.BCELoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)  # RMSprop, Adam

for epoch in range(15):  # loop over the dataset multiple times

    running_loss = 0.0
    correct = 0
    total = 0 
    for i, (images, labels) in enumerate(trainloader, 0): # get the inputs; data is a list of [images, labels]

        #images.shape -> [64, 10, 3, 224, 224]
        #labels.shape -> [64, 15]

        # zero the parameter gradients
        optimizer.zero_grad()

        images = images.cuda()

        #format input
        n_batches, n_crops, channels, height, width = images.size()
        image_batch = torch.autograd.Variable(images.view(-1, channels, height, width)) #640 images: 64 batches contain 10 crops each decomposed into 640 images

        labels = tile(labels, 0, 10).cuda() #duplicate for each crop the label [1,2],[3,4] => [1,2],[1,2],[3,4],[3,4] -> 640 labels

        # forward + backward + optimize
        outputs = model(image_batch)
        loss = criterion(outputs, labels.float())
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()

        correct += compute_score_with_logits(outputs, labels).sum()
        total += labels.size(0)

    print('Epoch: %d, loss: %.3f, Accuracy: %.3f' %
          (epoch + 1, running_loss, 100 * correct / total))

print('Finished Training')

  

[1,     1] loss: 0.000
Epoch: 1, loss: 195.437, Accuracy: 53.300
[2,     1] loss: 0.000
Epoch: 2, loss: 182.043, Accuracy: 54.332
[3,     1] loss: 0.000
Epoch: 3, loss: 177.248, Accuracy: 55.130
[4,     1] loss: 0.000
Epoch: 4, loss: 175.806, Accuracy: 55.634
[5,     1] loss: 0.000
Epoch: 5, loss: 173.292, Accuracy: 55.864
[6,     1] loss: 0.000
Epoch: 6, loss: 169.617, Accuracy: 57.492
[7,     1] loss: 0.000
Epoch: 7, loss: 166.166, Accuracy: 57.984
[8,     1] loss: 0.000
Epoch: 8, loss: 164.643, Accuracy: 58.312
[9,     1] loss: 0.000
Epoch: 9, loss: 159.967, Accuracy: 59.806
[10,     1] loss: 0.000
Epoch: 10, loss: 156.299, Accuracy: 61.204
[11,     1] loss: 0.000
Epoch: 11, loss: 151.776, Accuracy: 62.323
[12,     1] loss: 0.000
Epoch: 12, loss: 147.435, Accuracy: 63.601
[13,     1] loss: 0.000
Epoch: 13, loss: 144.000, Accuracy: 64.792
[14,     1] loss: 0.000
Epoch: 14, loss: 137.388, Accuracy: 67.204
[15,     1] loss: 0.000
Epoch: 15, loss: 131.393, Accuracy: 69.030
Finished Trai

In [0]:
model.eval()

In [37]:
correct = 0
total = 0
with torch.no_grad():
  for i, (images, labels) in enumerate(testloader, 0):
    images = images.cuda()
    n_batches, n_crops, channels, height, width = images.size()
    image_batch = torch.autograd.Variable(images.view(-1, channels, height, width))
    labels = tile(labels, 0, 10).cuda()
    outputs = model(image_batch)
    correct += compute_score_with_logits(outputs, labels).sum()
    total += labels.size(0)
    

print('Accuracy on test set: %.3f' % (100 * correct / total))


Accuracy on test set: 53.872
